### EDA of Apr 2021 kaggle TPS

This notebook is used to do initial exploration of the training data, get an understanding of the distribution and completeness of each column and relationship with the target. The aim is to build a broad understanding and make some baseline predictions.

Synthanic [Data Dictionary](https://www.kaggle.com/c/tabular-playground-series-apr-2021/data?select=train.csv)

In [ ]:
# imports for EDA
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic

In [ ]:
# imports for inference
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer

In [ ]:
sns.set_style("whitegrid")
sns.set_context("paper")

In [ ]:
df = pd.read_csv("../data/raw/train.csv")

In [ ]:
df.head()

We can get some idea of what to expect from the first few rows - next to look at the distribution of each more closely, and think about how to handle missing data and how to encode sex, class, port of embarkation etc.

In [ ]:
df.info()

Noting null values in Age, Ticket, Fare, Cabin and Embarked

In [ ]:
df.describe()

So with a 43% survival rate, hopefully we can do better than predicting everyone as lost at 57%. Noticing a lot of missing ages, not so many missing fares. 

For the purposes of this challend I'm assuming the passenger IDs are all ok - on to survival, and lots of plots.

In [ ]:
sns.countplot(data=df, x="Survived")
plt.show()

In [ ]:
sns.countplot(data=df, x="Pclass", hue="Survived")
plt.show()

In [ ]:
sns.countplot(data=df, x="Sex", hue="Survived")
plt.show()

No missing values here - but note significant differences in outcomes between male and female.

In [ ]:
df.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
mosaic(df, ['Pclass', 'Sex', 'Survived'])
plt.show()

In [ ]:
sns.displot(data=df, x="Age", hue="Survived", kind="kde")
plt.show()

In [ ]:
sns.countplot(data=df, x="SibSp", hue="Survived")
plt.show()

The SibSp chart is dominated by the number of passengers with 0 or 1 - let's zoom in on 2+

In [ ]:
dplot = sns.countplot(data=df, x="SibSp", hue="Survived")
dplot.set(xlim=(1.5, 6.5))
dplot.set(ylim=(0.0, 2100.0))
plt.show()

In [ ]:
sns.countplot(data=df, x="Parch", hue="Survived")
plt.show()

In [ ]:
dplot = sns.countplot(data=df, x="Parch", hue="Survived")
dplot.set(xlim=(0.5, 7.5))
dplot.set(ylim=(0, 8000))
plt.show()

Values (and the lack of) from 5+ in SibSp and 4+ in Parch may lead to overfitting in the unseen data - binning these into broader categories could be more robust

In [ ]:
print("Number of unique tickets: {}".format(df['Ticket'].nunique()))
print("Number of null tickets: {}".format(df['Ticket'].isnull().sum()))

In [ ]:
sns.displot(data=df, x="Fare", hue="Survived", kind="kde")

Those paying < 100 did not do so well. Let's take a closer look at passengers who paid more:

In [ ]:
dplot = sns.displot(data=df, x="Fare", hue="Survived", kind="kde")
dplot.set(xlim=(100, 800))
dplot.set(ylim=(0, 0.001))
plt.show()

In [ ]:
print("Number of passengers missing embarkation port: {}".format(df['Embarked'].isnull().sum()))

In [ ]:
emb_fill = {'Embarked': 'X'}
df.fillna(value=emb_fill, inplace=True)

In [ ]:
df.groupby(['Embarked', 'Survived'])['Survived'].count()

In [ ]:
sns.countplot(data=df, x="Embarked", hue="Survived")
plt.show()

In [ ]:
df['Cabin'].head(10)

In [ ]:
print("Number of unique cabin IDs: {}".format(df['Cabin'].nunique()))
print("Number of null cabin IDs: {}".format(df['Cabin'].isnull().sum()))

In [ ]:
df['CabinArea'] = df['Cabin'].fillna('X').map(lambda x: x[0].strip())

In [ ]:
sns.countplot(data=df, x="CabinArea", hue="Survived")
plt.show()

In [ ]:
df['Ticket'].head(10)

In [ ]:
print("Number of unique ticket IDs: {}".format(df['Ticket'].nunique()))
print("Number of null ticket IDs: {}".format(df['Ticket'].isnull().sum()))

In [ ]:
df['TicketStub'] = df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X').lower()

In [ ]:
df['TicketStub'].unique()

Clearly some cleaning to be done here - for example 'SC/PARIS', 'S.C./PARIS', 'SC/Paris' look similar. What sort of frequencies are we looking at here, is this worth pursuing?

In [ ]:
df.groupby(['TicketStub'], sort=False, dropna=True)['Survived'].count()

In [ ]:
df.groupby(['TicketStub'], sort=False, dropna=True)['Survived'].mean()

So, maybe? There is work to be done to work out the best way to impute missing data around Age, SibSp, Parch, Ticket, Cabin, and Embarked, but first I want to compare a quick mvp to a dummy benchmark to set a baseline. For this the categorical columns will need to be encoded, nulls filled with temporary junk, after breaking out to a train/test split.

In [ ]:
target = df[['Survived']]
features = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'CabinArea', 'TicketStub']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
categorical = ['Pclass', 'Sex', 'Embarked', 'CabinArea', 'TicketStub']
numeric = ['Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
X_train[numeric].describe()

In [ ]:
#sns.displot(data=X_train, x='Fare', kind='ecdf')
sns.displot(data=X_train, x='Fare', kind='kde')
plt.show()

In [ ]:
fare_map = X_train[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
X_train['Fare'] = X_train['Fare'].fillna(X_train['Pclass'].map(fare_map['Fare']))

In [ ]:
fare_map

Other ideas on handling missing data:
- X_train['Age'] = X_train.groupby(['Sex','Pclass'], sort=False)['Age'].apply(lambda x: x.fillna(x.median()))
- X_train['Fare'] = X_train.groupby(['Pclass'], sort=False)['Fare'].apply(lambda x: x.fillna(x.median()))

Turned out to be difficult to apply to the unseen data, the mapping works much better and is more explainable.

In [ ]:
sns.displot(data=X_train, x='Age', kind='kde')
plt.show()

In [ ]:
# can't quite get this to work - could be worth exploring later
draft_age_map = X_train[['Age', 'Sex', 'Pclass']].dropna().groupby(['Sex', 'Pclass']).median().to_dict()
draft_age_map

In [ ]:
age_map = X_train[['Age', 'Sex']].dropna().groupby(['Sex']).median().to_dict()
X_train['Age'] = X_train['Age'].fillna(X_train['Sex'].map(age_map['Age']))
age_map

In [ ]:
sns.displot(data=X_train, x='Age', kind='kde')

In [ ]:
X_train.info()

With nulls filled and a train/test split specified, numeric columns are scaled and categorical columns are encoded.

In [ ]:
categorical = ['Pclass', 'Sex', 'Embarked', 'CabinArea', 'TicketStub']
numeric = ['Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
preprocessor = make_column_transformer(
    (StandardScaler(), numeric),
    (OneHotEncoder(drop='if_binary'), categorical), 
    remainder='drop')

Initial estimator is logistic regression as a baseline for a binary classification task.

In [ ]:
model = make_pipeline(
    preprocessor,
    LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5))

_ = model.fit(X_train, np.ravel(y_train))

To set a floor for model performance, let's see a classification report if the ship is predicted as going down with no survivors.

In [ ]:
y_pred = model.predict(X_train)
y_dummy = np.full(shape=len(y_pred), fill_value=0)
print(classification_report(y_train, y_dummy))

In [ ]:
print(classification_report(y_train, y_pred))

Performance on training data is ok at 77% overall accuracy; better than guessing that no one makes it at least, at 57% accuracy. Compare to the held out test set:

In [ ]:
X_test.info()

In [ ]:
# using age and fare maps from X_train, no peeking
X_test['Fare'] = X_test['Fare'].fillna(X_test['Pclass'].map(fare_map['Fare']))
X_test['Age'] = X_test['Age'].fillna(X_test['Sex'].map(age_map['Age']))

In [ ]:
y_unseen = model.predict(X_test)
print(classification_report(y_test, y_unseen))

To finish with the same steps are carried out on the competition test set and submitted. With more time you could do EDA on the test set to look for drift across each category, but for right now I'm interested in how this model performs without any additional intervention.

This initial submission achieved a leaderboard score of 0.77000. 

In [ ]:
# there needs to be a proper CV grid search eventually - submitting this as is to kaggle yielded leaderboard score of 0.78642
model_gb = make_pipeline(
    preprocessor,
    GradientBoostingClassifier())

_ = model_gb.fit(X_train, np.ravel(y_train))

In [ ]:
y_pred_gb = model_gb.predict(X_train)
print(classification_report(y_train, y_pred_gb))

In [ ]:
y_unseen_gb = model_gb.predict(X_test)
print(classification_report(y_test, y_unseen_gb))

The ensemble method achieved a leaderboard score of 0.78642 out of the box. Let's see what a moderate parameter search yields

In [ ]:
no_param = {'random_state': [42],
            'solver': ['saga'],
            'penalty': ['elasticnet'],
            'l1_ratio': [0.5]} 

mini_param = {
    "loss":["deviance"], 
    "learning_rate": [0.2],
    "n_estimators": [250],
    "subsample": [0.8],
    "criterion": ["mse"],
    "max_depth": [15],
    "max_features": ["sqrt"],
    "random_state": [42],
    "validation_fraction": [0.1],
    "n_iter_no_change": [10]}

param_grid = {
    "loss":["deviance", "exponential"], 
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "n_estimators":[100, 250, 500],
    "subsample":[0.5, 0.8, 0.9, 1.0],
    "criterion": ["friedman_mse",  "mse"],
    "max_depth":[6, 9, 15],
    "max_features":["log2","sqrt"],
    "random_state":[42],
    "validation_fraction":[0.1],
    "n_iter_no_change":[5, 10]}

scores = {'accuracy': make_scorer(accuracy_score), 
          'f1': make_scorer(f1_score)}

In [ ]:
from sklearn.compose import ColumnTransformer

gbc = GradientBoostingClassifier()

pre_process = ColumnTransformer([
    ('num', StandardScaler(), numeric),
    ('cat', OneHotEncoder(), categorical)], n_jobs=-1)

gbc_grid = GridSearchCV(gbc, param_grid, scoring=scores, refit=False, cv=3, n_jobs=-1)
# gbc_grid.get_params().keys() for troubleshooting

In [ ]:
gbc_grid.get_params().keys()

In [ ]:
grid_out = pd.DataFrame.from_dict(gbc_grid.cv_results_)

In [ ]:
grid_out_loc = "../data/inference/grid_out.csv"
grid_out.to_csv(grid_out_loc)

In [ ]:
pd.set_option("display.max_columns", None)
grid_out.sort_values(['rank_test_f1','rank_test_accuracy'], ascending=[True, True]).head(5)

In [ ]:
grid_out.sort_values(['rank_test_accuracy','rank_test_f1'], ascending=[True, True]).head(5)

Now to take the parameters of the models with best f1 and accuracy for submission

In [ ]:
acc_param = {
    'criterion': 'mse', 
    'learning_rate': 0.05, 
    'loss': 'deviance', 
    'max_depth': 6, 
    'max_features': 'sqrt', 
    'n_estimators': 100, 
    'n_iter_no_change': 5, 
    'random_state': 42, 
    'subsample': 1.0, 
    'validation_fraction': 0.1}

f1_param = {
    'criterion': 'mse', 
    'learning_rate': 0.025, 
    'loss': 'exponential', 
    'max_depth': 9, 
    'max_features': 
    'log2', 
    'n_estimators': 250, 
    'n_iter_no_change': 5, 
    'random_state': 42, 
    'subsample': 1.0, 
    'validation_fraction': 0.1}

In [ ]:
clf_acc = GradientBoostingClassifier(**acc_param).fit(X_prep, np.ravel(y_train))

In [ ]:
y_pred_acc = clf_acc.predict(X_prep)
print(classification_report(y_train, y_pred_acc))

In [ ]:
X_test_prep = pre_process.transform(X_test)
y_test_acc = clf_acc.predict(X_test_prep)
print(classification_report(y_test, y_test_acc))

In [ ]:
clf_f1 = GradientBoostingClassifier(**f1_param).fit(X_prep, np.ravel(y_train))

In [ ]:
y_pred_f1 = clf_f1.predict(X_prep)
print(classification_report(y_train, y_pred_f1))

In [ ]:
y_test_f1 = clf_f1.predict(X_test_prep)
print(classification_report(y_test, y_test_f1))

Both of these models seem consistent with each other - I don't think much of an improvement can be expected on the held out kaggle set.

In [ ]:
# convert to code and run if running inference on test set for kaggle submission
file_acc = "../data/inference/cv_acc_gb.csv"
file_f1 = "../data/inference/cv_f1_gb.csv"

test_df = pd.read_csv("../data/raw/test.csv")
test_df.head()

In [ ]:
test_df.info()

In [ ]:
na_fill = {'Embarked': 'X', 'Cabin': 'X', 'Ticket': 'X'}
test_df.fillna(value=na_fill, inplace=True)
test_df['Cabin'] = test_df['Cabin'].map(lambda x: x[0].strip())
test_df['Ticket'] = test_df['Ticket'].map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')
test_df['Fare'] = test_df['Fare'].fillna(test_df['Pclass'].map(fare_map['Fare']))
test_df['Age'] = test_df['Age'].fillna(test_df['Sex'].map(age_map['Age']))
test_df.rename(columns={'Cabin': 'CabinArea', 'Ticket': 'TicketStub'}, inplace=True)
test_features = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'CabinArea', 'TicketStub']]
test_prep = pre_process.transform(test_features)

In [ ]:
test_f1 = clf_f1.predict(test_prep)
test_f1

In [ ]:
test_df['Survived'] = test_f1.tolist()
test_submission = test_df[['PassengerId', 'Survived']]
test_submission.to_csv(file_f1, index=False)

In [ ]:
test_acc = clf_acc.predict(test_prep)

In [ ]:
test_df['Survived'] = test_acc.tolist()
test_submission = test_df[['PassengerId', 'Survived']]
test_submission.to_csv(file_acc, index=False)

In [ ]:
test_df['f1'] = test_f1.tolist()
test_df['acc'] = test_acc.tolist()
test_df.drop(columns='Survived', inplace=True)

In [ ]:
test_df['diff'] = test_df['f1'] == test_df['acc']
test_df['diff'].value_counts()

Although they differ in 1638 cases, both f1 and accuracy GBM models score 0.80394 on the leaderboard - perhaps they differ on the 75% held back for evaluation after the challenge ends.

In any case, cracking 80% accuracy on the public leaderboard caused a jump of hundreds of places, so it was good to see some positive impact from a good old fashioned grid search